In [ ]:
#this notebook will show yearly variation through undertaken analysis on the same day across the date range of the PVGIS database
#the aim is to show the ranging/error bars for each route 

In [ ]:
#import relevant packaages 
import pandas.tseries.offsets as po
import pvlib
#This package is supplied in the main branch of Github
import Coords_2_EY
import pandas as pd 
import numpy as np      
import matplotlib.pyplot as plt
import timeit
from solcast_frames.latlng import LatLng
from solcast_frames.radiationframehandler import RadiationFrameHandler
from solcast_frames.powerframehandler import PowerFrameHandler
from datetime import datetime, timedelta, date
import solcast as sc
import requests
from dateutil import tz
#this package is supplied in the main Github branch`
import Trip_to_Distinct_Coords
import os
import geemap
import json
import openrouteservice
from openrouteservice import convert
from geojson import MultiLineString
from geojson import LineString
from timezonefinder import TimezoneFinder
import pytz
from pytz import timezone
import calendar

client = openrouteservice.Client(key = '5b3ce3597851110001cf6248d5cc6329856d4104b16de2d55d56f603')

<h2> Import Processed Data</h2>
This section imports the locations of Woolworths distribution centres and stores, the 3 predefined trips and their coordinates. To further save time, the PVGIS data is extracted from the web interface for each coordinate for each trip and supplied in the location specified.

In [ ]:
#imported processed woolworths stores and distribution centres data
ww_dcs = pd.read_csv("data/ww_dcs_points_ll.csv")
ww_stores = pd.read_csv("data/ww_stores_points_ll.csv")

In [ ]:
#import predefined coordinates to save processing time
#This trip is from Sydney Chilled Distribution Centre to Woolworths Town Hall in NSW
NSW_coords = pd.read_csv('data/Trip_Coords/NSW/SydChilled_TownHall.csv')
NSW_coords.set_index('Unnamed: 0', inplace = True)

In [ ]:
#This trip is from Woolworths Stud Park to Melbourne Chilled Distribution Centre
VIC_coords = pd.read_csv('data/Trip_Coords/VIC/StudPark_MelbChilled.csv')
VIC_coords.set_index('Unnamed: 0', inplace = True)

In [ ]:
#this trip is from Perth Chilled Distribution Centre to Woolworths Collie
WA_coords = pd.read_csv('data/Trip_Coords/WA/PerthChilled_Collie.csv')
WA_coords.set_index('Unnamed: 0', inplace = True)

In [ ]:
NSW_PVGIS_data = {}
for i in range(0, len(NSW_coords)-1):
    NSW_PVGIS_data[i] = pd.read_csv("data/PVGIS/Processed/NSW_PVGIS_processed_"+f"{i}")

In [ ]:
VIC_PVGIS_data = {}
for i in range(0, len(VIC_coords)-1):
    VIC_PVGIS_data[i] = pd.read_csv("data/PVGIS/Processed/VIC_PVGIS_processed_"+f"{i}")

In [ ]:
WA_PVGIS_data = {}
for i in range(0, len(WA_coords)-1):
    WA_PVGIS_data[i] = pd.read_csv("data/PVGIS/Processed/WA_PVGIS_processed_"+f"{i}")

<h2>Function Definition</h2>
Specify functions that will repeatedly be use throughout this coming analysis. Note that some adjustments are required here since I am working with offline PVGIS data rather than querying the API in every function call.

In [ ]:
def solar_data_processing(df):
    # the difference between this function and the one specified in the Coords_2_EY library
    # is accounting for the difference in variable type due to importing data from a csv file
    df['aoi'] = 90 - df['H_sun']
    df['IAM'] = [pvlib.iam.martin_ruiz(df['aoi'][i]) for i in range(0,len(df))]
    #Calculate total irradiance incident on panels
    # Used equation GHI = DNI * IAM * cos(H_sun) + DHI 
    df['TII'] = df['Gb(i)']*df['IAM']*np.cos(np.deg2rad(df['H_sun'].astype(float)))+df['Gd(i)']
    #calculate cell temperatures
    df['T_cell'] = pvlib.temperature.faiman(df['TII'], df['T2m'], wind_speed = df['WS10m'])
    df.drop(axis = 1, labels = 'index', inplace = True )
    return df

In [ ]:
def full_stage_downloaded(data,year,month,day,hour,minute,n_panels,P_stc,gamma_p):
    sol_data = Coords_2_EY.data_collation_monthly(year,month,day,hour,minute,data)
    processed_sol_data = solar_data_processing(sol_data)
    final = Coords_2_EY.calculate_power_energy(processed_sol_data,n_panels,P_stc,gamma_p)
    return final

In [ ]:
def annual_variation(data, state, month, hour, minute, n_panels, P_stc, gamma_p):
    if state  == 'WA':
        start = date(2006, month, 1)
        end = date(2017, 1, 1)
    else:
        start = date(2006, month, 2)
        end = date(2021, 1, 1)
    date_range = pd.date_range(start,end,freq='d').date
    res = []
    for i in date_range:
        if i.month == month:
            res.append(i)
    Dates = []
    energy_output = []
    for i in range(0,len(res)):
        year = res[i].year
        month = res[i].month
        day = res[i].day
        Dates.append(res[i])
        #print(year,month,day,hour,minute)
        table = full_stage_downloaded(data,year,month,day,hour,minute,n_panels,P_stc,gamma_p)
        energy_output.append(table['Energy'].sum())
        #print(f'{i}',"/100")
        #print("Current Time =", current_time)
        #print(res[i])
    outputs = pd.DataFrame()
    outputs.insert(0, "Date", Dates)
    outputs.insert(1, "Energy Output", energy_output)
    outputs.to_csv('results/monthlyvariation/'+f'{state}'+'/'+f'{hour}'+'.csv')
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Finished ", hour, " ", minute, "Current Time=", current_time)
    return energy_output

In [ ]:
def year_collated(data, state, hour, minute, n_panels, P_stc, gamma_p):
    yearly_data = []
    i = 1
    for i in range(1,13):
        res = annual_variation(data, state, i, hour, minute, n_panels, P_stc, gamma_p)
        yearly_data.append(res)
        i += 1
    return yearly_data

In [ ]:
def histogram_plot(dataset): 
    n_obs=len(dataset)
    intervals = round(np.sqrt(n_obs))
    rang = max(dataset) - min(dataset)
    width = rang/intervals 
    bins=[round(min(dataset))+round(width*i) for i in range(0,intervals)]    
    fig, ax = plt.subplots()
    n, bins, patches = ax.hist(dataset, bins)
    ax.set_xlabel('Energy Output (Wh)')
    ax.set_ylabel('Count')
    return fig, ax

In [ ]:
def stats_analysis(dataset,state,time,npanels):
    '''
    Simple descriptive statistics analysis of energy output by month data
    
    :params dataset: energy output aggregated by month
    :params state: a string referring to the state of Australia in which the trip occurs
    :params time: string of time of trip - simply for naming convention only
    :params npanels: number of solar panels also only for csv name
    
    :returns: a pandas dataframe with average of energy output, standard deviation, median and standard error
    '''
    months = list(calendar.month_name)
    months = months[1:]
    mean = []
    stdevs = []
    median = []
    sem = []
    df = pd.DataFrame(dataset).T
    for i in range(0, len(dataset)): 
        mean.append(df[i].mean())
        sem.append(df[i].sem())
        median.append(df[i].median())
        stdevs.append(df[i].std())
    stats = pd.DataFrame({'months': months, 'energy_output_mean_Wh': mean,'standard deviation': stdevs, 'energy_output_med_Wh': median, 'standard error': sem})
    stats.to_csv('results/monthlyvariation/'+f'{state}'+'/' + f'{time}'+'_'+f'{npanels}'+ '_stats.csv')
    

<h2>Collate yearly data for trips and times of interest</h2>

In [ ]:
NSW_morn_monthly = year_collated(NSW_PVGIS_data, 'NSW', 7, 30, 3, 400, -0.0029)
NSW_lunch_monthly = year_collated(NSW_PVGIS_data, 'NSW', 12, 30, 3, 400, -0.0029)
NSW_even_monthly = year_collated(NSW_PVGIS_data, 'NSW', 16, 30, 3, 400, -0.0029)

In [ ]:
months = list(calendar.month_name)
months = months[1:]
#save relevant data to annual_selection/state directory as a csv
collated_morn_NSW = pd.DataFrame(NSW_morn_monthly)
collated_morn_NSW = collated_morn_NSW.transpose()
collated_morn_NSW.columns = months
collated_morn_NSW.to_csv('data/annual_selection/NSW/morn_3.csv')
collated_lunch_NSW = pd.DataFrame(NSW_lunch_monthly)
collated_lunch_NSW = collated_lunch_NSW.transpose()
collated_lunch_NSW.columns = months
collated_lunch_NSW.to_csv('data/annual_selection/NSW/lunch_3.csv')
collated_even_NSW = pd.DataFrame(NSW_even_monthly)
collated_even_NSW = collated_even_NSW.transpose()
collated_even_NSW.columns = months
collated_even_NSW.to_csv('data/annual_selection/NSW/even_3.csv')

In [ ]:
VIC_morn_monthly = year_collated(VIC_PVGIS_data, 'VIC', 7, 30, 3, 400, -0.0029)
VIC_lunch_monthly = year_collated(VIC_PVGIS_data, 'VIC', 12, 30, 3, 400, -0.0029)
VIC_even_monthly = year_collated(VIC_PVGIS_data, 'VIC', 16, 30, 3, 400, -0.0029) 

In [ ]:
months = list(calendar.month_name)
months = months[1:]
collated_morn_VIC = pd.DataFrame(VIC_morn_monthly)
collated_morn_VIC = collated_morn_VIC.transpose()
collated_morn_VIC.columns = months
collated_morn_VIC.to_csv('data/annual_selection/VIC/morn_3.csv')
collated_lunch_VIC = pd.DataFrame(VIC_lunch_monthly)
collated_lunch_VIC = collated_lunch_VIC.transpose()
collated_lunch_VIC.columns = months
collated_lunch_VIC.to_csv('data/annual_selection/VIC/lunch_3.csv')
collated_even_VIC= pd.DataFrame(VIC_even_monthly)
collated_even_VIC = collated_even_VIC.transpose()
collated_even_VIC.columns = months
collated_even_VIC.to_csv('data/annual_selection/VIC/even_3.csv')

In [ ]:
WA_morn_monthly = year_collated(WA_PVGIS_data, 'WA', 7, 36, 3, 400, -0.0029)
WA_lunch_monthly = year_collated(WA_PVGIS_data, 'WA', 12, 36, 3, 400, -0.0029)
WA_even_monthly = year_collated(WA_PVGIS_data, 'WA', 16, 36, 3, 400, -0.0029)

In [ ]:
months = list(calendar.month_name)
months = months[1:]
collated_morn_WA = pd.DataFrame(WA_morn_monthly)
collated_morn_WA = collated_morn_WA.transpose()
collated_morn_WA.columns = months
collated_morn_WA.to_csv('data/annual_selection/WA/morn_3.csv')
collated_lunch_WA = pd.DataFrame(WA_lunch_monthly)
collated_lunch_WA = collated_lunch_WA.transpose()
collated_lunch_WA.columns = months
collated_lunch_WA.to_csv('data/annual_selection/WA/lunch_3.csv')
collated_even_WA= pd.DataFrame(WA_even_monthly)
collated_even_WA = collated_even_WA.transpose()
collated_even_WA.columns = months
collated_even_WA.to_csv('data/annual_selection/WA/even_3.csv')

In [ ]:
stats_analysis(VIC_morn_monthly,'VIC','morning',3)
stats_analysis(VIC_lunch_monthly,'VIC','lunch',3)
stats_analysis(VIC_even_monthly,'VIC','evening',3)
stats_analysis(NSW_morn_monthly,'NSW','morning',3)
stats_analysis(NSW_lunch_monthly,'NSW','lunch',3)
stats_analysis(NSW_even_monthly,'NSW','evening',3)
stats_analysis(WA_morn_monthly,'WA','morning',3)
stats_analysis(WA_lunch_monthly,'WA','lunch',3)
stats_analysis(WA_even_monthly,'WA','evening',3)

<h2>Import existing statistics for plotting</h2>
Only if analysis has already been undertaken and saved to correct directory

In [ ]:
VIC_morn_stats = pd.read_csv('results/monthlyvariation/VIC/morning_3_stats.csv')
VIC_lunch_stats = pd.read_csv('results/monthlyvariation/VIC/lunch_3_stats.csv')
VIC_even_stats = pd.read_csv('results/monthlyvariation/VIC/evening_3_stats.csv')

In [ ]:
NSW_morn_stats = pd.read_csv('results/monthlyvariation/NSW/morning_3_stats.csv')
NSW_lunch_stats = pd.read_csv('results/monthlyvariation/NSW/lunch_3_stats.csv')
NSW_even_stats = pd.read_csv('results/monthlyvariation/NSW/evening_3_stats.csv')

In [ ]:
WA_morn_stats = pd.read_csv('results/monthlyvariation/WA/morning_3_stats.csv')
WA_lunch_stats = pd.read_csv('results/monthlyvariation/WA/lunch_3_stats.csv')
WA_even_stats = pd.read_csv('results/monthlyvariation/WA/evening_3_stats.csv')

<h2>Plot Results</h2>
Results from annual analysis are plotted demonstrating average energy yield, and error bars with standard deviation.

In [ ]:
months = list(calendar.month_name)
months = months[1:]
fig = plt.subplots(figsize = (7,5))
barWidth = 0.25
br1 = np.arange(12)
br2 = [x+barWidth for x in br1]
br3 = [x+barWidth for x in br2]
plt.bar(br1, height = NSW_morn_stats['energy_output_mean_Wh'], color = '#003f5c', width = barWidth, label = 'NSW', yerr = NSW_morn_stats['standard deviation'], capsize = 3, ecolor = 'grey')
plt.bar(br2, height = VIC_morn_stats['energy_output_mean_Wh'], color = '#bc5090', width = barWidth, label = 'VIC', yerr = VIC_morn_stats['standard deviation'], capsize = 3, ecolor = 'grey')
plt.bar(br3, height = WA_morn_stats['energy_output_mean_Wh'], color = '#ffa600', width = barWidth, label = 'WA', yerr = WA_morn_stats['standard deviation'], capsize = 3, ecolor = 'grey')
plt.xlabel("Month")
plt.ylabel("Energy Yield (Wh)")
plt.title("Energy yield by month for each morning trip (7:30am)")
plt.xticks([r+barWidth for r in range(12)],months, rotation = 45)
plt.legend()
plt.savefig('results/monthlyvariation/graphs/seminar/morning_trip_3.png',bbox_inches='tight')

In [ ]:
fig = plt.subplots(figsize = (7,5))
barWidth = 0.25
br1 = np.arange(12)
br2 = [x+barWidth for x in br1]
br3 = [x+barWidth for x in br2]
plt.bar(br1, height = NSW_lunch_stats['energy_output_mean_Wh'], color = '#003f5c', width = barWidth, label = 'NSW', yerr = NSW_lunch_stats['standard deviation'], capsize = 3, ecolor = 'grey')
plt.bar(br2, height = VIC_lunch_stats['energy_output_mean_Wh'], color = '#bc5090', width = barWidth, label = 'VIC', yerr = VIC_lunch_stats['standard deviation'], capsize = 3, ecolor = 'grey')
plt.bar(br3, height = WA_lunch_stats['energy_output_mean_Wh'], color = '#ffa600', width = barWidth, label = 'WA', yerr = WA_lunch_stats['standard deviation'], capsize = 3, ecolor = 'grey')
plt.xlabel("Month")
plt.ylabel("Energy Yield (Wh)")
plt.title("Energy yield by month for each lunch trip (12:30pm)")
plt.xticks([r+barWidth for r in range(12)],months, rotation = 45)
plt.legend()
plt.savefig('results/monthlyvariation/graphs/seminar/lunch_trip_3.png',bbox_inches='tight')

In [ ]:
fig = plt.subplots(figsize = (7,5))
barWidth = 0.25
br1 = np.arange(12)
br2 = [x+barWidth for x in br1]
br3 = [x+barWidth for x in br2]
plt.bar(br1, height = NSW_even_stats['energy_output_mean_Wh'], color = '#003f5c', width = barWidth, label = 'NSW', yerr = NSW_even_stats['standard deviation'], capsize = 3, ecolor = 'grey' )
plt.bar(br2, height = VIC_even_stats['energy_output_mean_Wh'], color = '#bc5090', width = barWidth, label = 'VIC', yerr = VIC_even_stats['standard deviation'], capsize = 3, ecolor = 'grey')
plt.bar(br3, height = WA_even_stats['energy_output_mean_Wh'], color = '#ffa600', width = barWidth, label = 'WA', yerr = WA_even_stats['standard deviation'], capsize = 3, ecolor = 'grey')
plt.xlabel("Month")
plt.ylabel("Energy Yield (Wh)")
plt.title("Energy yield by month for each evening trip (4:30pm)")
plt.xticks([r+barWidth for r in range(12)],months, rotation = 45)
plt.legend()
plt.ylim([0,1650])
plt.savefig('results/monthlyvariation/graphs/seminar/evening_trip_3.png',bbox_inches='tight')